In [1]:
# Importar librerías
import os
import pandas as pd
import numpy as np
import boto3
import datetime as dt
from src.utils import (
    read_configuration,
    obtener_info_banxico,
    obtener_inflacion_inegi
)

# config yaml
global_config = read_configuration()

In [2]:
token_banxico = global_config.get('etl').get('token_banxico')
token_inegi = global_config.get('etl').get('token_inegi')

In [3]:
# get data
tipo_de_cambio = obtener_info_banxico('SF43718', '2000-01-01', '2024-04-01', token_banxico)
tiie_28_dias = obtener_info_banxico('SF43783', '2000-01-01', '2024-04-01', token_banxico)
inflación = obtener_inflacion_inegi(token_inegi)

In [4]:
# save data
tipo_de_cambio.to_csv('data/raw/tipo_de_cambio.csv',index=False)
tiie_28_dias.to_csv('data/raw/tasa_de_interes.csv',index=False)
inflación.to_csv('data/raw/inflacion.csv',index=False)

In [5]:
# session with AWS
aws_profile = global_config.get('main').get('aws_profile')
session = boto3.Session(profile_name=aws_profile)
s3 = session.client('s3')
# upload data
BUCKET_NAME = global_config.get('etl').get('aws_bucket')
s3.upload_file(Filename="data/raw/inflacion.csv", Bucket=BUCKET_NAME, Key="econ/raw/inflacion/inflacion.csv")
s3.upload_file(Filename="data/raw/tasa_de_interes.csv", Bucket=BUCKET_NAME, Key="econ/raw/tasa_de_interes/tasa_de_interes.csv")
s3.upload_file(Filename="data/raw/tipo_de_cambio.csv", Bucket=BUCKET_NAME, Key="econ/raw/tipo_de_cambio/tipo_de_cambio.csv")
